In [0]:
%%sh


In [0]:
%%sh
aws s3 ls s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-08-11/
#aws s3 ls s3://prod.appannie.mktint.data/market_log_fact/granularity=DAY/date=2018-08-11/country_code=AU/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.pubapp-fact.v1/fact/granularity=weekly/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-fact.v1/fact/granularity=monthly/date=2018-08-31/

In [0]:

spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-08-11/country_code=AU/").rdd.takeSample(False, 10, seed=0)[0]

In [0]:

#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp-publisher-fact.v1/fact/granularity=weekly/date=2018-08-11/").show(1)
spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-08-25").filter('platform_id = 2 and device_id = 1').limit(1).collect()

In [0]:

delete = spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-08-11").filter('ad_app_publisher_SOV is null').count()
all = spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-08-11").count()
expected = all - delete
print(expected)
actual = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp-publisher-fact.v1/fact/granularity=weekly/date=2018-08-11/").count()
expected == actual

In [0]:

delete = spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK_TO_DATE/date=2018-08-04/").filter('creative_NETCNT is null and creative_NETLST is null and creative_PUBCNT is null and creative_PUBLST is null and creative_STOCNT is null and creative_STOLST is null and creative_SOV is null and creative_ad_app_SOV is null').count()
all = spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK_TO_DATE/date=2018-08-04/").count()
expected = all - delete
print(expected)
actual = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-fact.v1/fact/granularity=weekly/date=2018-08-04/").count()
print(actual)
expected == actual

In [0]:

unified_samples = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp-publisher-fact.v1/fact/granularity=weekly/date=2018-08-11/device_code=ios-phone").rdd.takeSample(False, 10, seed=0)
for i in range(10):
    raw_sample = spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-08-11/country_code={}/".format(unified_samples[i]['country_code'])).filter('platform_id = 2 and device_id = 1 and ad_app_id = {} and network_id = {} and ad_app_publisher_id = {} and ad_app_publisher_SOV is not null'.format(unified_samples[i]['ad_app_id'], unified_samples[i]['ad_platform_id'], unified_samples[i]['publisher_id'])).collect()
    print(raw_sample[0]['ad_app_publisher_SOV'] == unified_samples[i]['ad_app_share_of_publisher'])


In [0]:

#spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-08-11/").filter('platform_id = 2 and device_id = 1 and network_id = 1 and creative_id = "411c464601df5dd9bf5c526443a64ca0"').count()
spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-08-11/").filter('platform_id = 2 and device_id = 1 and creative_id = "427616627d5e0b2d443327081b42f988"').limit(10).collect()
#wtd = spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK_TO_DATE/date=2018-08-11/").filter('platform_id = 2 and device_id = 1 and creative_id = "411c464601df5dd9bf5c526443a64ca0" and network_id = 142 and country_code = "US"').limit(10).collect()

In [0]:

spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK_TO_DATE/date=2018-08-11/country_code=CN/").filter('platform_id = 2 and device_id = 1 and creative_id = "427616627d5e0b2d443327081b42f988"').limit(10).collect()

In [0]:

spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-fact.v1/fact/granularity=weekly/date=2018-08-11/device_code=ios-phone").filter('creative_id = 90100000428421 and country_code = "CN"').collect()

In [0]:

CREATIVE_ADAPP_FACT_METRIC = ('creative_NETCNT', 'creative_NETLST', 'creative_PUBCNT', 'creative_PUBLST', 'creative_STOCNT', 'creative_STOLST', 'creative_SOV', 'creative_ad_app_SOV')
unified_samples = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-fact.v1/fact/granularity=weekly/date=2018-08-11/device_code=ios-phone").rdd.takeSample(False, 10, seed=0)
for i in range(10):
    raw_creative_id = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-id-mapping.v1/dimension/_partition=_default_partition/").filter('creative_id = {}'.format(unified_samples[i]['creative_id'])).collect()[0]['creative_md5']
    raw_sample_wtd= spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK_TO_DATE/date=2018-08-11/country_code={}/".format(unified_samples[i]['country_code'])).filter('platform_id = 2 and device_id = 1 and network_id = {} and ad_app_id = "{}" and creative_id = "{}"'.format(unified_samples[i]['ad_platform_id'], unified_samples[i]['ad_app_id'], raw_creative_id)).filter('creative_NETCNT is not null or creative_NETLST is not null or creative_PUBCNT is not null or creative_PUBLST is not null or creative_STOCNT is not null or creative_STOLST is not null').collect()
    raw_sample_week = spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-08-11/country_code={}/".format(unified_samples[i]['country_code'])).filter('platform_id = 2 and device_id = 1 and network_id = {} and ad_app_id = "{}" and creative_id = "{}"'.format(unified_samples[i]['ad_platform_id'], unified_samples[i]['ad_app_id'], raw_creative_id)).filter('creative_SOV is not null or creative_ad_app_SOV is not null').collect()
    print(unified_samples[i]['creative_id'])
    print(raw_creative_id)
    print(unified_samples[i]['country_code'])
    print(raw_sample_wtd)
    print(raw_sample_week)
    print(raw_sample_wtd[0][CREATIVE_ADAPP_FACT_METRIC[0]] == unified_samples[i]['ad_platform_count'])
    print(raw_sample_wtd[0][CREATIVE_ADAPP_FACT_METRIC[1]] == unified_samples[i]['ad_platform_list'])
    print(raw_sample_wtd[0][CREATIVE_ADAPP_FACT_METRIC[2]] == unified_samples[i]['pub_app_count'])
    print(raw_sample_wtd[0][CREATIVE_ADAPP_FACT_METRIC[3]] == unified_samples[i]['pub_app_list'])
    print(raw_sample_wtd[0][CREATIVE_ADAPP_FACT_METRIC[4]] == unified_samples[i]['country_count'])
    print(raw_sample_wtd[0][CREATIVE_ADAPP_FACT_METRIC[5]] == unified_samples[i]['country_list'])
    if raw_sample_week:
        print(raw_sample_week[0][CREATIVE_ADAPP_FACT_METRIC[6]] == unified_samples[i]['creative_share_of_ad_platform'])
        print(raw_sample_week[0][CREATIVE_ADAPP_FACT_METRIC[7]] == unified_samples[i]['creative_share_of_advertiser'])
    else:
        print(unified_samples[i]['creative_share_of_ad_platform'] == None)
        print(unified_samples[i]['creative_share_of_advertiser'] == None)

In [0]:

unified_samples = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp-pubapp-fact.v1/fact/granularity=weekly/date=2018-08-11/device_code=ios-phone").rdd.takeSample(False, 10, seed=0)
for i in range(10):
    raw_sample = spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-08-11/country_code={}/".format(unified_samples[i]['country_code'])).filter('platform_id = 2 and device_id = 1 and ad_app_id = {} and network_id = {} and pub_app_id = {}'.format(unified_samples[i]['ad_app_id'], unified_samples[i]['ad_platform_id'], unified_samples[i]['pub_app_id'])).filter('ad_app_pub_app_SOV is not null or pub_app_ad_app_SOV is not null or pub_app_ad_app_NETCNT is not null or pub_app_ad_app_NETLST is not null').collect()
    print(raw_sample[0]['ad_app_pub_app_SOV'] == unified_samples[i]['ad_app_share_of_pub_app'])
    print(raw_sample[0]['pub_app_ad_app_SOV'] == unified_samples[i]['pub_app_share_of_ad_app'])
    print(raw_sample[0]['pub_app_ad_app_NETCNT'] == unified_samples[i]['ad_platform_count'])
    print(raw_sample[0]['pub_app_ad_app_NETLST'] == unified_samples[i]['ad_platform_list'])

In [0]:

#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp-pubapp-fact.v1/fact/granularity=weekly/date=2018-08-11/device_code=ios-phone").rdd.takeSample(False, 10, seed=0)
#spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK_TO_DATE/date=2018-08-11/").show(1)
spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-08-11/").filter('creative_SOV is not null or creative_ad_app_SOV is not null').select("creative_id", "creative_SOV", "creative_ad_app_SOV").show(10)
#spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-08-11/country_code=TH/").filter('platform_id = 2 and device_id = 1 and ad_app_id = "880047117" and network_id = 1 and pub_app_id = "1087664350"').filter('ad_app_pub_app_SOV is not null or pub_app_ad_app_SOV is not null or pub_app_ad_app_NETCNT is not null or pub_app_ad_app_NETLST is not null').collect()

In [0]:

from aaintdatapipeline.core.pipeline import schema, type_
from aaintdatapipeline.core.utils.identifier import virtual_id
from aaintdatapipeline.core.fs.driver.unified import (
    Query,
    UnifiedDTO,
    exist,
    read)


urn = Urn(
    identifier=virtual_id(),
    manipulation=type_.ManipulationType.INSERT,
    namespace='app-tech.store.app-est-category-rank.v1',
    data_type=type_.DataType.METRIC
)

urn.query = Query(
    wheres=[
        {
            "data_type": u"metric",
            "date": [u'2018-11-30'],
            "_identifier": 1
        }
    ]
)

print(type(unified.read(spark, urn)))


In [0]:

spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-adapp-visibility.v1/").show(1)

In [0]:
%%sh
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp-pubapp-visibility.v1/fact/date=2019-06-18/
#aws s3 ls s3://prod.appannie.mktint.data/market_pubapp_adapp_visibility_fact/granularity=DAY/date=2019-06-18/
aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-category-rank.v1/fact/granularity=weekly/date=2018-07-14/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app.v1/dimension/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app-est-category-rank.v1/fact/granularity=weekly/date=2018-07-14/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp-pubapp-visibility.v1/fact/date=2019-06-18/
#aws s3 ls s3://prod.appannie.mktint.data/market_pubapp_adapp_visibility_fact/granularity=DAY/date=2019-06-18/


In [0]:

#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp-pubapp-visibility.v1/fact/date=2019-06-18/").filter('country_code = "WW" and ad_app_id = "20600001570347" and pub_app_id = "20600000000014" and ad_platform_id = 1').show(1)
#print spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp-pubapp-visibility.v1/fact/date=2019-06-18/").count()
#spark.read.parquet("s3://prod.appannie.mktint.data/market_pubapp_adapp_visibility_fact/granularity=DAY/date=2019-06-18/").show(1)
print spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-fact.v1/fact/granularity=weekly/").schema.names

In [0]:

#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app.v1/dimension/").filter('id = "539179365"').limit(1).collect()
#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-category-rank.v1/fact/granularity=weekly/date=2018-07-14/").show(1)
spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2018-07-14/").filter('creative_id is not null').show(1)
#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app.v1/dimension/").filter('id = "539179365"').schema.names
#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app-est-category-rank.v1/fact/granularity=weekly/date=2018-07-14/").show(1)
#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-fact.v1/fact/granularity=weekly/").filter('creative_id = "90100000000829"').show()
#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-id-mapping.v1/dimension/").filter('creative_md5 = "a3a4a20ac20ab47cf3221c7e8a6d7f49"').collect()

In [0]:

spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-fact.v1/fact/granularity=weekly/").limit(1).collect()
#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.pubapp-fact.v1/fact/granularity=weekly/").show(1)
#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp-pubapp-fact.v1/fact/granularity=weekly/").show(1)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-ad_app_estimate.v1/fact/granularity=monthly/device_id=1001/store_id=1/




In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-ad_app_estimate.v1/fact/").count()


In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-ad_app_estimate.v1/fact/granularity=monthly/device_id=1001/store_id=1/").limit(1).collect()


In [0]:
%%sh


In [0]:
%%sh

ls ~/bdp/application/aaintdatapipeline/core/conf/
cat ~/bdp/application/aaintdatapipeline/core/conf/settings_local.py



In [0]:
%%sh
